In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("./BartSpektro")
# import modeling_bart
# import BartSpektro

In [2]:
%pip install rdkit

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)


import argparse
from datetime import datetime
import json
import os
import pickle
import random
import time
# import wandb
from pathlib import Path
import gc
import glob
from tqdm import tqdm

import numpy as np
from transformers import TrainingArguments, Trainer, BartConfig, BartForConditionalGeneration
# from transformers.file_utils import logging
# from tensorboardX import SummaryWriter
import torch

# custom veci
from dataset import SpectroDataset, SpectroDataCollator
sys.path.append('data')
sys.path.append('bart_spektro')
from modeling_bart_spektro import BartSpektoForConditionalGeneration
from configuration_bart_spektro import BartSpektroConfig
from data_preprocess1 import print_args
from bart_spektro_tokenizer import BartSpektroTokenizer
from tokenizers import Tokenizer
from transformers import PreTrainedTokenizerFast

# from utils import add_special_tokens, generate_sample, sample_seq, set_seed, top_k_top_p_filtering, print_args

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
SEQ_LEN=200
config = BartSpektroConfig(vocab_size = 600,
                             max_position_embeddings = SEQ_LEN,
                             encoder_layers = 12,
                             encoder_ffn_dim = 4096,
                             encoder_attention_heads = 16,
                             decoder_layers = 12,
                             decoder_ffn_dim = 4096,
                             decoder_attention_heads = 16,
                             encoder_layerdrop = 0.0,
                             decoder_layerdrop = 0.0,
                             activation_function = 'gelu',
                             d_model = 1024,
                             dropout = 0.2,
                             attention_dropout = 0.0,
                             activation_dropout = 0.0,
                             init_std = 0.02,
                             classifier_dropout = 0.0,
                             scale_embedding = False,
                             use_cache = True,
                             num_labels = 3,
                             pad_token_id = 501,
                             bos_token_id = 503,
                             eos_token_id = 502,
                             is_encoder_decoder = True,
                             decoder_start_token_id = 502,
                             forced_eos_token_id = 502,
                             max_log_id=9,
                          )

In [5]:
sbart = BartSpektoForConditionalGeneration(config)

In [6]:
device = "cpu"

def generate_smiles(inputs, model, top_k=200, top_p=0.8, do_sample=True, num_beams=1, temperature=1):
    with torch.no_grad():
#         outputs = model(input_ids = inputs["input_ids"].unsqueeze(0).to(device=device), 
#                         position_ids = inputs["position_ids"].unsqueeze(0).to(device=device),
#                         labels = inputs["labels"].unsqueeze(0).to(device=device))
        generated = model.generate(
                       input_ids=inputs["input_ids"].unsqueeze(0).to(device=device), 
                       position_ids=inputs["position_ids"].unsqueeze(0).to(device=device),
#                        position_ids=None,
                        top_p=top_p,
                       top_k=top_k,
    #                    min_length=20,
    #                    max_length=200,
                       do_sample=do_sample,
                       num_beams=num_beams,
                       temperature=temperature
                       ).tolist()[0]
        return inputs, generated

In [8]:
%pip install trl
import torch
# from transformers import GPT2Tokenizer
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model
from trl.core import respond_to_batch

# 1. load a pretrained model
model = AutoModelForCausalLMWithValueHead(sbart)
model_ref = AutoModelForCausalLMWithValueHead(sbart)

tokenizer_type = "_bbpe_1M" # for spektro tokenizer use ""
tokenizer = PreTrainedTokenizerFast(tokenizer_file=f"./tokenizer/bbpe_tokenizer/bart{tokenizer_type}_tokenizer.model")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# 2. initialize trainer
ppo_config = {'batch_size': 1, }
config = PPOConfig(**ppo_config)
ppo_trainer = PPOTrainer(config, model, model_ref, tokenizer)

# 3. encode a query
query_txt = ""
# query_tensor = tokenizer.encode(query_txt, return_tensors="pt")

query_tensor = torch.tensor([14, 19, 25, 26, 27], dtype=int, device=device).unsqueeze(0)

# 4. generate model response
response_tensor  = torch.tensor(generate_smiles({"input_ids":torch.tensor([14, 19, 25, 26, 27], dtype=int), "position_ids": torch.tensor([1, 1, 2, 6, 2], dtype=int)}, model=model)[1], dtype=int).unsqueeze(0)
# response_txt = tokenizer.decode(response_tensor)

# print(response_tensor[1])

# 5. define a reward for response
# (this could be any reward such as human feedback or output from another model)
reward = [torch.tensor(1.0)]

print(response_tensor[0].dtype)

# 6. train model with ppo
train_stats = ppo_trainer.step([query_tensor[0].type(torch.int)], [response_tensor[0]], reward)

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 43 kB 472 kB/s eta 0:00:011
     |████████████████████████████████| 469 kB 9.5 MB/s eta 0:00:01
     |████████████████████████████████| 199 kB 23.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 65.0 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 75.7 MB/s eta 0:00:01
     |████████████████████████████████| 35.0 MB 75.3 MB/s eta 0:00:01
     |████████████████████████████████| 145 kB 72.5 MB/s eta 0:00:01
     |████████████████████████████████| 110 kB 36.3 MB/s eta 0:00:01
     |████████████████████████████████| 213 kB 70.7 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 2.0 MB/s  eta 0:00:01
     |████████████████████████████████| 262 kB 62.3 MB/s eta 0:00:01
     |████████████████████████████████| 161 kB 63.4 MB/s eta 0:00:01
     |████████████████████████████████| 195 kB 54.1 MB/s eta 0:00:01
     |█████████████████████

/opt/conda/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:104.)
  return torch._C._cuda_getDeviceCount() > 0
/storage/brno2/home/ahajek/.local-Pytorch-21.SIF/lib/python3.8/site-packages/trl/trainer/ppo_trainer.py:210: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(
/storage/brno2/home/ahajek/.local-Pytorch-21.SIF/lib/python3.8/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens

torch.int64


AttributeError: 'torch.Size' object has no attribute 'to'